In [1]:
import numpy as np
import pandas as pd
import csv

## Import of datas

In [2]:
#clean dataset
data = pd.read_csv('clean_dataset.csv')

In [3]:
data.head()

,Unnamed: 0,ETER ID Year,ETER ID,National identifier,English Institution Name,Country Code,Region of establishment (NUTS 2),Students enrolled at ISCED 8 - men,Students enrolled at ISCED 8 - women,Students enrolled ISCED 8 - gender unclassified,...,Personnel expenditure (EURO),Non-personnel expenditure (EURO),Expenditure unclassified (EURO),Total Current expenditure (EURO),Capital expenditure (EURO),Basic government allocation (EURO),Total Current revenues (EURO),Research active institution,Included in official R&D statistics,R&D Expenditure (EURO)
0,1178,ES0001.2016,ES0001,None,University of Almeria,ES,ES61,374,440,0,...,None,None,None,None,None,None,None,1,1,None
1,1179,ES0002.2016,ES0002,None,University of CÃ¡diz,ES,ES61,468,403,0,...,None,None,None,None,None,None,None,1,1,None
2,1180,ES0003.2016,ES0003,None,University of Cordoba,ES,ES61,615,640,0,...,None,None,None,None,None,None,None,1,1,None
3,1181,ES0004.2016,ES0004,None,University of Granada,ES,ES61,1462,1267,0,...,None,None,None,None,None,None,None,1,1,None
4,1182,ES0005.2016,ES0005,None,University of Huelva,ES,ES61,240,180,0,...,None,None,None,None,None,None,None,1,1,None


In [4]:
data.shape

(1929, 38)

## Calcul of R4 : Personnel expenditure (EURO) / Total Current expenditure (EURO

In [5]:
perso_exp = data['Personnel expenditure (EURO)']
total_exp = data['Total Current expenditure (EURO)']

In [6]:
data['R4']=np.zeros((len(data),1))

for i in range(perso_exp.shape[0]):
    if  perso_exp.iloc[i]=='None' or total_exp.iloc[i]=='None':
        data['R4'].iloc[i]= np.nan
    else :
        num=pd.to_numeric(perso_exp.iloc[i])
        denom=pd.to_numeric(total_exp.iloc[i])
        data['R4'].iloc[i]= num/denom

/Users/robertapassarelli/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
# Sort ratios (All HEIs) and select the level that identify the top and bottom 5% of ratio values
data = data.sort_values(by = 'R4')
Xlratio = data["R4"].quantile(q = 0.05)
Xuratio = data["R4"].quantile(q = 0.95)

In [8]:
print(Xlratio)
print(Xuratio)

0.5207530493318115
0.7925022615870028


In [ ]:
# or 2%
# Xlratio = data["R4"].quantile(q = 0.02)
# Xuratio = data["R4"].quantile(q = 0.98)

# or 10 %
# Xlratio = data["R4"].quantile(q = 0.1)
# Xuratio = data["R4"].quantile(q = 0.9)

In [9]:
# Define sub-distribution for the analysis by country and for each sub-distribution sort ratios ascendent

data_IT = data[data['Country Code'] == 'IT']
data_IT.shape
data_IT = data_IT.sort_values(by = 'R4')

data_FI = data[data['Country Code'] == 'FI']
data_FI.shape
data_FI = data_IT.sort_values(by = 'R4')

data_NO = data[data['Country Code'] == 'NO']
data_NO.shape
data_NO = data_IT.sort_values(by = 'R4')

data_SE = data[data['Country Code'] == 'SE']
data_SE.shape
data_SE = data_IT.sort_values(by = 'R4')

data_PT = data[data['Country Code'] == 'PT']
data_PT.shape
data_PT = data_IT.sort_values(by = 'R4')

data_ES = data[data['Country Code'] == 'ES']
data_ES.shape
data_ES = data_IT.sort_values(by = 'R4')

In [10]:
# For each sub-distribution, we identify the cases where the ratio are below our Xlratio and Xuratio
def lower(data_country):
    data_country['R4_lower']=np.zeros((len(data_country),1))
    for i in range(len(data_country['R4'])):
        if data_country['R4'].iloc[i] < Xlratio:
            data_country['R4_lower'].iloc[i] = 1
        else:
            data_country['R4_lower'].iloc[i] = 0
    return(data_country)

def upper(data_country):
    data_country['R4_upper']=np.zeros((len(data_country),1))
    for i in range(len(data_country['R4'])):
        if data_country['R4'].iloc[i] > Xuratio:
            data_country['R4_upper'].iloc[i] = 1
        else:
            data_country['R4_upper'].iloc[i] = 0
    return(data_country)
            
def alarm(data_country):
    data_country['alarm']=np.zeros((len(data_country),1))
    for i in range(len(data_country['R4_lower'])):
            if data_country['R4_lower'].iloc[i] == 1 or data_country['R4_upper'].iloc[i] == 1:
                data_country['alarm'].iloc[i] = True
            else:
                data_country['alarm'].iloc[i]= False
    return(data_country)

data_IT = lower(data_IT)
data_IT =  upper(data_IT)
data_IT = alarm(data_IT) 

data_FI = lower(data_FI)
data_FI = upper(data_FI)
data_FI = alarm(data_FI) 

data_NO = lower(data_NO)
data_NO = upper(data_NO)
data_NO = alarm(data_NO) 

data_SE = lower(data_SE)
data_SE = upper(data_SE)
data_SE = alarm(data_SE) 

data_PT = lower(data_PT)
data_PT = upper(data_PT)
data_PT = alarm(data_PT) 

data_ES = lower(data_ES)
data_ES = upper(data_ES)
data_ES = alarm(data_ES)

In [11]:
# We count the number of values which are true in order to identify the percentage of alarm cases
nb_alarm_true_IT = data_IT['alarm'].value_counts()[1]
nb_alarm_true_FI = data_FI['alarm'].value_counts()[1]
nb_alarm_true_NO = data_NO['alarm'].value_counts()[1]
nb_alarm_true_SE = data_SE['alarm'].value_counts()[1]
nb_alarm_true_PT = data_PT['alarm'].value_counts()[1]
nb_alarm_true_ES = data_ES['alarm'].value_counts()[1]

nb_alarm_true = nb_alarm_true_IT + nb_alarm_true_FI + nb_alarm_true_NO + nb_alarm_true_SE + nb_alarm_true_PT + nb_alarm_true_ES
R4_final = round((nb_alarm_true / len(data))*100,2)
print('With our level which identify the top and bottom of ratio values, we have ', R4_final, '% of alarm cases for R4')

With our level which identify the top and bottom of ratio values, we have  16.17 % of alarm cases for R4


## Calcul of R5 : Total Current expenditure (EURO) / Total Current revenues (EURO)


In [12]:
total_exp = data['Total Current expenditure (EURO)']
total_rev = data['Total Current revenues (EURO)']

In [13]:
data['R5']=np.zeros((1929,1))

for i in range(total_exp.shape[0]):
    if  total_exp.iloc[i]=='None' or total_rev.iloc[i]=='None':
        data['R5'].iloc[i]=0
    else :
        num=pd.to_numeric(total_exp.iloc[i])
        denom=pd.to_numeric(total_rev.iloc[i])
        data['R5'].iloc[i]= num/denom        

In [14]:
# Sort ratios (All HEIs) and select the level that identify the top and bottom 5% of ratio values
data = data.sort_values(by = 'R5')
Xlratio = data["R5"].quantile(q = 0.05)
Xuratio = data["R5"].quantile(q = 0.95)

In [ ]:
# or 2%
# Xlratio = data["R5"].quantile(q = 0.02)
# Xuratio = data["R5"].quantile(q = 0.98)

# or 10 %
# Xlratio = data["R5"].quantile(q = 0.1)
# Xuratio = data["R5"].quantile(q = 0.9)

In [15]:
# Define sub-distribution for the analysis by country and for each sub-distribution sort ratios ascendent

data_IT = data[data['Country Code'] == 'IT']
data_IT.shape
data_IT = data_IT.sort_values(by = 'R5')

data_FI = data[data['Country Code'] == 'FI']
data_FI.shape
data_FI = data_IT.sort_values(by = 'R5')

data_NO = data[data['Country Code'] == 'NO']
data_NO.shape
data_NO = data_IT.sort_values(by = 'R5')

data_SE = data[data['Country Code'] == 'SE']
data_SE.shape
data_SE = data_IT.sort_values(by = 'R5')

data_PT = data[data['Country Code'] == 'PT']
data_PT.shape
data_PT = data_IT.sort_values(by = 'R5')

data_ES = data[data['Country Code'] == 'ES']
data_ES.shape
data_ES = data_IT.sort_values(by = 'R5')

In [16]:
# For each sub-distribution, we identify the cases where the ratio are below our Xlratio and Xuratio

def lower(data_country):
    data_country['R5_lower']=np.zeros((len(data_country),1))
    for i in range(len(data_country['R5'])):
        if data_country['R5'].iloc[i] < Xlratio:
            data_country['R5_lower'].iloc[i] = 1
        else:
            data_country['R5_lower'].iloc[i] = 0
    return(data_country)

def upper(data_country):
    data_country['R5_upper']=np.zeros((len(data_country),1))
    for i in range(len(data_country['R5'])):
        if data_country['R5'].iloc[i] > Xuratio:
            data_country['R5_upper'].iloc[i] = 1
        else:
            data_country['R5_upper'].iloc[i] = 0
    return(data_country)
            
def alarm(data_country):
    data_country['alarm']=np.zeros((len(data_country),1))
    for i in range(len(data_country['R5_lower'])):
            if data_country['R5_lower'].iloc[i] == 1 or data_country['R5_upper'].iloc[i] == 1:
                data_country['alarm'].iloc[i] = True
            else:
                data_country['alarm'].iloc[i]= False
    return(data_country)

data_IT = lower(data_IT)
data_IT =  upper(data_IT)
data_IT = alarm(data_IT) 

data_FI = lower(data_FI)
data_FI = upper(data_FI)
data_FI = alarm(data_FI) 

data_NO = lower(data_NO)
data_NO = upper(data_NO)
data_NO = alarm(data_NO) 

data_SE = lower(data_SE)
data_SE = upper(data_SE)
data_SE = alarm(data_SE) 

data_PT = lower(data_PT)
data_PT = upper(data_PT)
data_PT = alarm(data_PT) 

data_ES = lower(data_ES)
data_ES = upper(data_ES)
data_ES = alarm(data_ES)

In [17]:
# We count the number of values which are true in order to identify the percentage of alarm cases

nb_alarm_true_IT = data_IT['alarm'].value_counts()[1]
nb_alarm_true_FI = data_FI['alarm'].value_counts()[1]
nb_alarm_true_NO = data_NO['alarm'].value_counts()[1]
nb_alarm_true_SE = data_SE['alarm'].value_counts()[1]
nb_alarm_true_PT = data_PT['alarm'].value_counts()[1]
nb_alarm_true_ES = data_ES['alarm'].value_counts()[1]

nb_alarm_true = nb_alarm_true_IT + nb_alarm_true_FI + nb_alarm_true_NO + nb_alarm_true_SE + nb_alarm_true_PT + nb_alarm_true_ES

R5_final = round((nb_alarm_true / len(data))*100,2)
print('With our level which identify the top and bottom of ratio values, we have ', R5_final, '% of alarm cases for R5')

With our level which identify the top and bottom of ratio values, we have  11.82 % of alarm cases for R5


## Calcul of R8 : Total graduates at ISCED 8 / Total students enrolled at ISCED 8

In [18]:
total_grad8 = data['Total graduates at ISCED 8']
total_enr = data['Total students enrolled at ISCED 8']

In [19]:
data['R8']=np.zeros((1929,1))

for i in range(total_grad8.shape[0]):
    if  total_grad8.iloc[i]=='None' or total_enr.iloc[i]=='None':
        data['R8'].iloc[i]=0
    else :
        num=pd.to_numeric(total_grad8.iloc[i])
        denom=pd.to_numeric(total_enr.iloc[i])
        data['R8'].iloc[i]= num/denom

/Users/robertapassarelli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/Users/robertapassarelli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in long_scalars
  if __name__ == '__main__':


In [20]:
# Sort ratios (All HEIs) and select the level that identify the top and bottom 5% of ratio values
data = data.sort_values(by = 'R8')
Xlratio = data["R8"].quantile(q = 0.05)
Xuratio = data["R8"].quantile(q = 0.95)

In [ ]:
#or 2%
# Xlratio = data["R8"].quantile(q = 0.02)
# Xuratio = data["R8"].quantile(q = 0.98)

#or 10 %
# Xlratio = data["R8"].quantile(q = 0.1)
# Xuratio = data["R8"].quantile(q = 0.9)

In [21]:
# Define sub-distribution for the analysis, i.e. by country and sort them

data_IT = data[data['Country Code'] == 'IT']
data_IT.shape
data_IT = data_IT.sort_values(by = 'R8')

data_FI = data[data['Country Code'] == 'FI']
data_FI.shape
data_FI = data_IT.sort_values(by = 'R8')

data_NO = data[data['Country Code'] == 'NO']
data_NO.shape
data_NO = data_IT.sort_values(by = 'R8')

data_SE = data[data['Country Code'] == 'SE']
data_SE.shape
data_SE = data_IT.sort_values(by = 'R8')

data_PT = data[data['Country Code'] == 'PT']
data_PT.shape
data_PT = data_IT.sort_values(by = 'R8')

data_ES = data[data['Country Code'] == 'ES']
data_ES.shape
data_ES = data_IT.sort_values(by = 'R8')

In [22]:
# For each sub-distribution, we identify the cases where the ratio are below our Xlratio and Xuratio

def lower(data_country):
    data_country['R8_lower']=np.zeros((len(data_country),1))
    for i in range(len(data_country['R5'])):
        if data_country['R8'].iloc[i] < Xlratio:
            data_country['R8_lower'].iloc[i] = 1
        else:
            data_country['R8_lower'].iloc[i] = 0
    return(data_country)

def upper(data_country):
    data_country['R8_upper']=np.zeros((len(data_country),1))
    for i in range(len(data_country['R8'])):
        if data_country['R8'].iloc[i] > Xuratio:
            data_country['R8_upper'].iloc[i] = 1
        else:
            data_country['R8_upper'].iloc[i] = 0
    return(data_country)
            
def alarm(data_country):
    data_country['alarm']=np.zeros((len(data_country),1))
    for i in range(len(data_country['R8_lower'])):
            if data_country['R8_lower'].iloc[i] == 1 or data_country['R8_upper'].iloc[i] == 1:
                data_country['alarm'].iloc[i] = True
            else:
                data_country['alarm'].iloc[i]= False
    return(data_country)

data_IT = lower(data_IT)
data_IT =  upper(data_IT)
data_IT = alarm(data_IT) 

data_FI = lower(data_FI)
data_FI = upper(data_FI)
data_FI = alarm(data_FI) 

data_NO = lower(data_NO)
data_NO = upper(data_NO)
data_NO = alarm(data_NO) 

data_SE = lower(data_SE)
data_SE = upper(data_SE)
data_SE = alarm(data_SE) 

data_PT = lower(data_PT)
data_PT = upper(data_PT)
data_PT = alarm(data_PT) 

data_ES = lower(data_ES)
data_ES = upper(data_ES)
data_ES = alarm(data_ES)

In [23]:
nb_alarm_true_IT = data_IT['alarm'].value_counts()[1]
nb_alarm_true_FI = data_FI['alarm'].value_counts()[1]
nb_alarm_true_NO = data_NO['alarm'].value_counts()[1]
nb_alarm_true_SE = data_SE['alarm'].value_counts()[1]
nb_alarm_true_PT = data_PT['alarm'].value_counts()[1]
nb_alarm_true_ES = data_ES['alarm'].value_counts()[1]

nb_alarm_true = nb_alarm_true_IT + nb_alarm_true_FI + nb_alarm_true_NO + nb_alarm_true_SE + nb_alarm_true_PT + nb_alarm_true_ES

R8_final = round((nb_alarm_true / len(data))*100,2)
print('With our level which identify the top and bottom of ratio values, we have ', R8_final, '% of alarm cases for R8')

With our level which identify the top and bottom of ratio values, we have  17.73 % of alarm cases for R8
